# Philip Toolan, C17433026

#### Import dataset

In [1]:
import pandas as pd

df = pd.read_csv('./data/dataset.csv', header=None, na_values = [' ?'])

#### Insert headings of the dataset

In [2]:
headings = []

with open('./data/feature_names.txt', 'r') as fo:
    for line in fo:
        if line !='\n':
            headings.append(line.strip())
df.columns = headings

#### Create shortcuts for each column 

In [3]:
age = df['age']
work = df['workclass']
fnlwgt = df['fnlwgt']
edu = df['education']
e_num = df['education-num']
mar = df['marital-status']
occ = df['occupation']
rel = df['relationship']
race = df['race']
sex = df['sex']
cap_g = df['capital-gain']
cap_l = df['capital-loss']
hrs = df['hours-per-week']
nc = df['native-country']
tar = df['target']

#### Get description of each data type to figure out if it is continuous or categorical

In [4]:
age.describe()

count    30940.000000
mean        38.560763
std         13.639403
min         17.000000
25%         28.000000
50%         37.000000
75%         48.000000
max         90.000000
Name: age, dtype: float64

In [5]:
work.describe()

count        29205
unique           8
top        Private
freq         21576
Name: workclass, dtype: object

In [6]:
fnlwgt.describe()

count    3.094000e+04
mean     1.897864e+05
std      1.054064e+05
min      1.228500e+04
25%      1.178490e+05
50%      1.783840e+05
75%      2.373180e+05
max      1.484705e+06
Name: fnlwgt, dtype: float64

In [7]:
edu.describe()

count        30940
unique          16
top        HS-grad
freq          9976
Name: education, dtype: object

In [8]:
e_num.describe()

count    30940.000000
mean        10.081254
std          2.569967
min          1.000000
25%          9.000000
50%         10.000000
75%         12.000000
max         16.000000
Name: education-num, dtype: float64

In [9]:
mar.describe()

count                   30940
unique                      7
top        Married-civ-spouse
freq                    14201
Name: marital-status, dtype: object

In [10]:
occ.describe()

count               29198
unique                 14
top        Prof-specialty
freq                 3932
Name: occupation, dtype: object

In [11]:
rel.describe()

count        30940
unique           6
top        Husband
freq         12496
Name: relationship, dtype: object

In [12]:
race.describe()

count      30940
unique         5
top        White
freq       26442
Name: race, dtype: object

In [13]:
sex.describe()

count     30940
unique        2
top        Male
freq      20705
Name: sex, dtype: object

In [14]:
cap_g.describe()

count    30940.000000
mean      1081.812993
std       7443.773041
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max      99999.000000
Name: capital-gain, dtype: float64

In [15]:
cap_l.describe()

count    30940.000000
mean        86.569974
std        401.706023
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max       4356.000000
Name: capital-loss, dtype: float64

In [16]:
hrs.describe()

count    30940.000000
mean        40.408920
std         12.336945
min          1.000000
25%         40.000000
50%         40.000000
75%         45.000000
max         99.000000
Name: hours-per-week, dtype: float64

In [17]:
nc.describe()

count              30386
unique                41
top        United-States
freq               27719
Name: native-country, dtype: object

In [18]:
tar.describe()

count      30940
unique         2
top        <=50K
freq       23506
Name: target, dtype: object

## Automate for all the continuos features

In [19]:
# Had problems using the variable names from earlier
#con_feat = age, fnlwgt, cap_g, cap_l, hrs

con_feat = ['age', 'fnlwgt', 'capital-gain', 'capital-loss', 'hours-per-week']

con_header = ['Count', '% Miss', 'Card.', 'Min.', '1st Qrt.', 'Mean', 'Median', '3rd Qrt.', 'Max.', 'Std. Dev.']

con_df = pd.DataFrame(index=con_feat, columns=con_header)
con_df.index.name = 'FEATURENAME'
columns = df[con_feat]


#Max
con_df[con_header[8]] = columns.max()

#Min
con_df[con_header[3]] = columns.min()

#Standard deviation (Standard Deviation is rounded to 1 like in notes)
con_df[con_header[9]] = round(columns.std(), 1)

#Count
con_df[con_header[0]] = columns.count()

#First quartile
con_df[con_header[4]] = columns.quantile(0.25)

#Third quartile
con_df[con_header[7]] = columns.quantile(0.75)

#Cardinality
con_df[con_header[2]] = columns.nunique()

#Mean (Mean is rounded to 1 like in notes)
con_df[con_header[5]] = round(columns.mean(), 1)

#Median
con_df[con_header[6]] = columns.median()

#Percentage of values missing
missing = columns.isna()
con_df[con_header[1]] = missing.sum() / len(columns)

## Automate for all Categorical Features

In [20]:
cat_feat = ['workclass', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country', 'target']

cat_header = ['Count', '% Miss', 'Card.', 'Mode', 'Mode Freq.', 'Mode %', '2nd Mode', '2nd Mode Freq.', '2nd Mode %']

cat_df = pd.DataFrame(index=cat_feat, columns=cat_header)
cat_df.index.name = 'FEATURENAME'
columns = df[cat_feat]


#Count
cat_df[cat_header[0]] = columns.count()

#Used later in the for loop to get mode %
count = columns.count()

#Percentage of values missing
missing = columns.isna()
cat_df[cat_header[1]] = round((missing.sum() / len(columns)) * 100, 1)

#Cardinality
cat_df[cat_header[2]] = columns.nunique()

#Preparing arrays to store data, multiply by amount to give the arrays the correct length
amount = len(cat_feat)
mode_f = [''] * amount
modes = [''] * amount
modes_2 = [''] * amount
mode_p = [''] * amount
mode_f2 = [''] * amount
mode_p2 = [''] * amount
    
#Have to use for loop for modes as the index was not being handled correctly in my original idea
for col in columns:
    values = columns[col].value_counts()
    index = cat_feat.index(col)
    
    #Mode
    mode = values.index[0]
    modes[index] = mode

    #Mode Freq.
    mode_count = values.loc[mode]
    mode_f[index] = mode_count
    
    #Mode %
    mode_per = (mode_count/(count[index])) * 100
    mode_p[index] = round(mode_per, 1)
    
    #Mode 2
    mode_2 = values.index[1]
    modes_2[index] = mode_2
    
    #Mode 2 Freq.
    mode_count2 = values.loc[mode_2]
    mode_f2[index] = mode_count2
    
    #Mode 2 %
    mode_per2 = (mode_count2/(count[index]))*100
    mode_p2[index] = round(mode_per2, 1)
    
#assign the values 
cat_df[cat_header[3]] = modes
cat_df[cat_header[4]] = mode_f
cat_df[cat_header[5]] = mode_p
cat_df[cat_header[6]] = modes_2
cat_df[cat_header[7]] = mode_f2
cat_df[cat_header[8]] = mode_p2

# Original idea, needed to do a loop to complete correctly
#Mode
#cat_df[cat_header[3]] = columns.value_counts().index[0]
#print(columns.value_counts().index[0])
#Mode Freq.
#cat_df[cat_header[4]] = columns.value_counts()[0]
#print(columns.value_counts()[0])
#Mode %
#2nd Mode
#cat_df[cat_header[6]] = columns.value_counts().index[1]
#print(columns.value_counts().index[1])
#2nd Mode Freq.
#cat_df[cat_header[7]] = columns.value_counts()[1]
#print (columns.value_counts()[1])
#2nd Mode %

# Output

In [21]:
con_df.to_csv("./C17433026CONT.csv")

In [22]:
cat_df.to_csv("./C17433026CAT.csv")